# Função de Perda/Objetivo

A escolha da função objetivo está diretamente relacionada ao problema sendo resolvido. Assim como a camada de saída da rede.

Os principais problemas resolvidos por redes neurais são: Classificação e regressão.

No caso de regressão, para cálculo de preço de imóveis por exemplo, podemos definir uma função objetivo, ou função de perda da seguinte forma:

    Sendo y o valor real e y' o valor predito pela rede neural:
    
    Função de perda = |y' - y|     -> Distância absoluta (Perda L1)
    ou
    Função de perda = |y' - y|²    -> Distância quadrática (Mean Squared Error - MSE)

Dessa forma, o objetivo é minimizar a distâcia entre os dois valores. Já para problemas de classificação, uma função bastante utilizada é a **Cross-Entropy**, na qual vai atribuir pesos para os erros e acertos.

### Função de perda/objetivo para Classificação

#### Preparação dos dados e da rede neural

In [40]:
import torch
import pandas as pd
from torch import nn
from sklearn import datasets

In [41]:
# definição do device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cpu


In [42]:
# definição dos dados
# dataset de vinhos do sklearn

vinhos = datasets.load_wine()

dados = vinhos.data
dados_nomes = vinhos.feature_names

labels = vinhos.target
labels_nomes = vinhos.target_names

print("Formato dados:", dados.shape)
print("Formato labels:", labels.shape)

Formato dados: (178, 13)
Formato labels: (178,)


In [43]:
vinhos_df = pd.DataFrame(data=dados, columns=dados_nomes)
vinhos_df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [67]:
# Definição da rede neural

class WineClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(WineClassifier, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)    ## Camada hidden com 
        self.relu = nn.ReLU()                               ## Ativação ReLU
        self.out = nn.Linear(hidden_size, out_size)         ## Saída linear
        self.softmax = nn.Softmax(dim=-1)                         ## softmax para transformar em probabilidades

    def forward(self, dados):
        feature = self.relu(self.hidden(dados))     ## feature é a ativação ReLU da camada hidden
        output = self.softmax(self.out(feature))    ## softmax da camada de saída recebendo feature
        return output

In [45]:
input_size = dados.shape[1]  ## quantidade de features do dataset
hidden_size = 32            ## quantidade julgada necessária para o problema (hiperparametro)
output_size = len(labels_nomes) ## quantidade de tipos de vinhos

print("Input_size:", input_size)
print("Hidden_size:", hidden_size)
print("Output_size:", output_size)

Input_size: 13
Hidden_size: 32
Output_size: 3


In [46]:
# Criação da rede:

net = WineClassifier(input_size, hidden_size, output_size)
#net = net.to(device) ## cast para a GPU
net

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)

In [47]:
# função objetivo Cross Entropy Loss:

criterio = nn.CrossEntropyLoss()
#criterio = criterio.to(device) ## cast para a gpu

In [48]:
# Transformação dos dados em tensores

xtns = torch.from_numpy(dados).float() ## conversao para float32 ao invés do float64 padrao
ytns = torch.from_numpy(labels)

#xtns = xtns.to(device)
#ytns = ytns.to(device)

print(xtns.dtype)

torch.float32


In [68]:
previsao = net(xtns)
print("Formato da saída com as 3 probabilidades softmax:", previsao.shape)
print(previsao)

Formato da saída com as 3 probabilidades softmax: torch.Size([442, 1])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


In [50]:
perda = criterio(previsao, ytns)
print("Média de perda nas previsoes:", perda.data)

Média de perda nas previsoes: tensor(1.2818)


### Função de perda/objetivo para Regressão

#### Preparação dos dados e da rede neural

In [51]:
# definição dos dados
# dataset de diabetes do sklearn

diabetes = datasets.load_diabetes()

dados = diabetes.data
labels = diabetes.target

dados_nomes = diabetes.feature_names

print("Formato dados:", dados.shape)
print("Formato labels:", labels.shape)
print("Features:", dados_nomes)

Formato dados: (442, 10)
Formato labels: (442,)
Features: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']


In [52]:
diabetes_df = pd.DataFrame(data=dados, columns=dados_nomes)
diabetes_df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [69]:
# Definição da rede neural

class DiabetesRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(DiabetesRegressor, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)    ## Camada hidden com 
        self.relu = nn.ReLU()                               ## Ativação ReLU
        self.out = nn.Linear(hidden_size, out_size)         ## Saída linear
        self.softmax = nn.Softmax(dim=-1)                         ## softmax para transformar em probabilidades

    def forward(self, dados):
        feature = self.relu(self.hidden(dados))     ## feature é a ativação ReLU da camada hidden
        output = self.softmax(self.out(feature))    ## softmax da camada de saída recebendo feature
        return output

In [70]:
input_size = dados.shape[1]     ## quantidade de features do dataset
hidden_size = 32                ## quantidade julgada necessária para o problema (hiperparametro)
output_size = 1                 ## dimensão da saída (progressão da diabetes)

print("Input_size:", input_size)
print("Hidden_size:", hidden_size)
print("Output_size:", output_size)

Input_size: 10
Hidden_size: 32
Output_size: 1


In [71]:
# Criação da rede:

net = DiabetesRegressor(input_size, hidden_size, output_size)
#net = net.to(device) ## cast para a GPU
net

DiabetesRegressor(
  (hidden): Linear(in_features=10, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=1, bias=True)
  (softmax): Softmax(dim=-1)
)

In [72]:
# função objetivo MSE Loss:

criterio = nn.MSELoss()

In [73]:
# Transformação dos dados em tensores

xtns = torch.from_numpy(dados).float() ## conversao para float32 ao invés do float64 padrao
ytns = torch.from_numpy(labels).float()

print(xtns.shape, ytns.shape)

torch.Size([442, 10]) torch.Size([442])


In [75]:
previsoes = net(xtns)

# para a função MSELoss os dados e labels devem
# ter a mesma dimensionalidade, então fazemos um squeeze 
# nas previsoes para retirar uma dimensao e ficar igual
# ao ytns
perda = criterio(previsoes.squeeze(), ytns)
print("Média da distância quadrática entre o dado e o label:", perda.data)

Média da distância quadrática entre o dado e o label: tensor(28771.2148)
